In [268]:
import numpy as np

import seaborn as sns
sns.set_style("white")
sns.set_context("talk",font_scale=1.5, rc={"lines.linewidth": 2.5})

In [269]:
def simulation(a=0.5, d1=0.8, d2=0.9, endTime=100):
    """
    two server / Single Queue / Exponential Rates
    inputs:
        - a: mean of the interarrival time 
        - d1: mean of the service times of server 1
        - d2: mean of the service times of server 2
        - end: max simulation time (minutes)
    outputs:
        - numArr: number of Arrivals 
        - AvgTimeS: average time in System
    """
    # state variables 
    N = 0 # number of people in system
    N1 = 0 # number of people in server 1
    N2 = 0 # number of people in server 2
    t = 0 # simulation time (in minutes)
    FEL_time = [t+np.random.exponential(a),endTime]
    FEL_eventType = ['A','E']

    # metric tracking
    numArr = 0 
    TotalTimeS = 0 
    Debug = True
    threeOverTime = 0
    server1Busy = 0
    server1and2Busy = 0

    eventType = ''
    while(eventType != 'E'):
        if len(FEL_time) == 0:
            print('Error: FEL is empty. Ending simulation now')
            break
        
        nextEvent = np.argmin(FEL_time)
        eventTime = FEL_time.pop(nextEvent)
        eventType = FEL_eventType.pop(nextEvent)
        TotalTimeS += N*(eventTime-t)
        
        # total number of minutes where the system has three or more customers
        threeOverTime += (1 if N>=3 else 0)*(eventTime-t)#two customers with server, one customer in line
        server1Busy += (1 if N1>=1 else 0)*(eventTime-t)#proportion time server 1 is busy
        server1and2Busy += (1 if N1>=1 and N2>=1 else 0)*(eventTime-t)#proportion time both server busy
        
        t = eventTime
        
        

        #arrival 
        if eventType == 'A':
            N+=1 
            numArr+=1
            t_A = np.random.exponential(a)
            FEL_time.append(t+t_A)
            FEL_eventType.append('A')
            if N == 1 or (N == 2 and N1 == 0):
                t_S1 = np.random.exponential(d1)
                FEL_time.append(t+t_S1)
                FEL_eventType.append('D1')
                N1 += 1
            elif N == 2 and N2 == 0:
                t_S2 = np.random.exponential(d2)
                FEL_time.append(t+t_S2)
                FEL_eventType.append('D2')
                N2 += 1
        #departure from server 1
        elif eventType == 'D1':
            N-=1
            N1 -= 1
            if N >= 2:
                t_S1 = np.random.exponential(d1)
                FEL_time.append(t+t_S1)
                FEL_eventType.append('D1')
                N1 += 1
        #departure from server 2        
        elif eventType == 'D2':
            N-=1
            N2 -= 1
            if N >= 2:
                t_S2 = np.random.exponential(d2)
                FEL_time.append(t+t_S2)
                FEL_eventType.append('D2')
                N2 += 1
        
        if Debug == True:
            print("eventType: ",eventType," time: ","{:.2f}".format(eventTime), " N: ",N, "N1: ", N1, "N2: ",N2, 
                  " numArr: ",numArr," TotalTimeS: ","{:.2f}".format(TotalTimeS)," FEL: ",printFEL(FEL_time,FEL_eventType), "three: ", threeOverTime)

    return numArr, TotalTimeS / numArr, threeOverTime, server1Busy, server1and2Busy

In [270]:
def printFEL(FEL_time,FEL_eventType):
    strOutput=""
    for eventIndex in range(len(FEL_time)):
        strOutput += "("
        strOutput += str(FEL_eventType[eventIndex])
        strOutput += ","
        strOutput += "{:.2f}".format(FEL_time[eventIndex])
        strOutput += ") "
    return strOutput

In [271]:
numArr, AvgTimeS, three, one, two = simulation()


eventType:  A  time:  0.69  N:  1 N1:  1 N2:  0  numArr:  1  TotalTimeS:  0.00  FEL:  (E,100.00) (A,0.75) (D1,1.60)  three:  0.0
eventType:  A  time:  0.75  N:  2 N1:  1 N2:  1  numArr:  2  TotalTimeS:  0.07  FEL:  (E,100.00) (D1,1.60) (A,0.88) (D2,0.99)  three:  0.0
eventType:  A  time:  0.88  N:  3 N1:  1 N2:  1  numArr:  3  TotalTimeS:  0.31  FEL:  (E,100.00) (D1,1.60) (D2,0.99) (A,1.06)  three:  0.0
eventType:  D2  time:  0.99  N:  2 N1:  1 N2:  1  numArr:  3  TotalTimeS:  0.66  FEL:  (E,100.00) (D1,1.60) (A,1.06) (D2,2.01)  three:  0.11725263651219131
eventType:  A  time:  1.06  N:  3 N1:  1 N2:  1  numArr:  4  TotalTimeS:  0.79  FEL:  (E,100.00) (D1,1.60) (D2,2.01) (A,3.69)  three:  0.11725263651219131
eventType:  D1  time:  1.60  N:  2 N1:  1 N2:  1  numArr:  4  TotalTimeS:  2.41  FEL:  (E,100.00) (D2,2.01) (A,3.69) (D1,3.81)  three:  0.659320908995962
eventType:  D2  time:  2.01  N:  1 N1:  1 N2:  0  numArr:  4  TotalTimeS:  3.24  FEL:  (E,100.00) (A,3.69) (D1,3.81)  three:  0.

eventType:  D2  time:  54.07  N:  9 N1:  1 N2:  1  numArr:  89  TotalTimeS:  238.62  FEL:  (E,100.00) (A,54.44) (D1,55.45) (D2,54.17)  three:  30.308750659609935
eventType:  D2  time:  54.17  N:  8 N1:  1 N2:  1  numArr:  89  TotalTimeS:  239.47  FEL:  (E,100.00) (A,54.44) (D1,55.45) (D2,54.95)  three:  30.403439892135573
eventType:  A  time:  54.44  N:  9 N1:  1 N2:  1  numArr:  90  TotalTimeS:  241.61  FEL:  (E,100.00) (D1,55.45) (D2,54.95) (A,55.44)  three:  30.670541169589733
eventType:  D2  time:  54.95  N:  8 N1:  1 N2:  1  numArr:  90  TotalTimeS:  246.23  FEL:  (E,100.00) (D1,55.45) (A,55.44) (D2,57.11)  three:  31.183766829800156
eventType:  A  time:  55.44  N:  9 N1:  1 N2:  1  numArr:  91  TotalTimeS:  250.18  FEL:  (E,100.00) (D1,55.45) (D2,57.11) (A,56.47)  three:  31.67782150556689
eventType:  D1  time:  55.45  N:  8 N1:  1 N2:  1  numArr:  91  TotalTimeS:  250.21  FEL:  (E,100.00) (D2,57.11) (A,56.47) (D1,55.99)  three:  31.680801244217406
eventType:  D1  time:  55.99  N

eventType:  A  time:  97.15  N:  4 N1:  1 N2:  1  numArr:  170  TotalTimeS:  555.54  FEL:  (E,100.00) (D1,98.68) (D2,97.69) (A,97.38)  three:  65.89334889932165
eventType:  A  time:  97.38  N:  5 N1:  1 N2:  1  numArr:  171  TotalTimeS:  556.45  FEL:  (E,100.00) (D1,98.68) (D2,97.69) (A,97.47)  three:  66.11947715213064
eventType:  A  time:  97.47  N:  6 N1:  1 N2:  1  numArr:  172  TotalTimeS:  556.90  FEL:  (E,100.00) (D1,98.68) (D2,97.69) (A,97.57)  three:  66.21094516869341
eventType:  A  time:  97.57  N:  7 N1:  1 N2:  1  numArr:  173  TotalTimeS:  557.51  FEL:  (E,100.00) (D1,98.68) (D2,97.69) (A,98.18)  three:  66.31141665193266
eventType:  D2  time:  97.69  N:  6 N1:  1 N2:  1  numArr:  173  TotalTimeS:  558.35  FEL:  (E,100.00) (D1,98.68) (A,98.18) (D2,98.14)  three:  66.4314958982786
eventType:  D2  time:  98.14  N:  5 N1:  1 N2:  1  numArr:  173  TotalTimeS:  561.02  FEL:  (E,100.00) (D1,98.68) (A,98.18) (D2,99.53)  three:  66.87739841310437
eventType:  A  time:  98.18  N:  

In [272]:
print("The number of arrivals is:",numArr)
print("The average time spent in the system is:",AvgTimeS)
print("The total time the system spent with one or more customer in queue:",three)
print("The proportion time that the server 1 is busy:",one)
print("The proportion time that the both server are busy:",two)

The number of arrivals is: 183
The average time spent in the system is: 3.1567033462558465
The total time the system spent with one or more customer in queue: 68.73979912512972
The proportion time that the server 1 is busy: 88.35535453206609
The proportion time that the both server are busy: 77.55736588847823


In [273]:
def sim():
    th=[] 
    on=[] 
    tw = []
    for i in range(20):
        numArr, AvgTimeS, three, one, two = simulation()
        th.append(three)
        on.append(one)
        tw.append(two)
    return th, on, tw
        
        

In [274]:
import numpy as np
th, on, tw = sim()

print("The total time the system spent with one or more customer in queue:",sum(th)/20)
print("The proportion time that the server 1 is busy:",sum(on)/20)
print("The proportion time that the both server are busy:", sum(tw)/20)

eventType:  A  time:  0.92  N:  1 N1:  1 N2:  0  numArr:  1  TotalTimeS:  0.00  FEL:  (E,100.00) (A,1.05) (D1,1.33)  three:  0.0
eventType:  A  time:  1.05  N:  2 N1:  1 N2:  1  numArr:  2  TotalTimeS:  0.13  FEL:  (E,100.00) (D1,1.33) (A,2.78) (D2,3.04)  three:  0.0
eventType:  D1  time:  1.33  N:  1 N1:  0 N2:  1  numArr:  2  TotalTimeS:  0.68  FEL:  (E,100.00) (A,2.78) (D2,3.04)  three:  0.0
eventType:  A  time:  2.78  N:  2 N1:  1 N2:  1  numArr:  3  TotalTimeS:  2.14  FEL:  (E,100.00) (D2,3.04) (A,3.14) (D1,3.42)  three:  0.0
eventType:  D2  time:  3.04  N:  1 N1:  1 N2:  0  numArr:  3  TotalTimeS:  2.65  FEL:  (E,100.00) (A,3.14) (D1,3.42)  three:  0.0
eventType:  A  time:  3.14  N:  2 N1:  1 N2:  1  numArr:  4  TotalTimeS:  2.75  FEL:  (E,100.00) (D1,3.42) (A,4.12) (D2,4.45)  three:  0.0
eventType:  D1  time:  3.42  N:  1 N1:  0 N2:  1  numArr:  4  TotalTimeS:  3.30  FEL:  (E,100.00) (A,4.12) (D2,4.45)  three:  0.0
eventType:  A  time:  4.12  N:  2 N1:  1 N2:  1  numArr:  5  Tot

eventType:  D1  time:  40.80  N:  2 N1:  1 N2:  1  numArr:  67  TotalTimeS:  89.36  FEL:  (E,100.00) (A,41.04) (D2,45.45) (D1,42.45)  three:  13.001672275111826
eventType:  A  time:  41.04  N:  3 N1:  1 N2:  1  numArr:  68  TotalTimeS:  89.82  FEL:  (E,100.00) (D2,45.45) (D1,42.45) (A,42.51)  three:  13.001672275111826
eventType:  D1  time:  42.45  N:  2 N1:  1 N2:  1  numArr:  68  TotalTimeS:  94.05  FEL:  (E,100.00) (D2,45.45) (A,42.51) (D1,42.86)  three:  14.413385945481384
eventType:  A  time:  42.51  N:  3 N1:  1 N2:  1  numArr:  69  TotalTimeS:  94.18  FEL:  (E,100.00) (D2,45.45) (D1,42.86) (A,42.52)  three:  14.413385945481384
eventType:  A  time:  42.52  N:  4 N1:  1 N2:  1  numArr:  70  TotalTimeS:  94.22  FEL:  (E,100.00) (D2,45.45) (D1,42.86) (A,42.79)  three:  14.427977234143949
eventType:  A  time:  42.79  N:  5 N1:  1 N2:  1  numArr:  71  TotalTimeS:  95.27  FEL:  (E,100.00) (D2,45.45) (D1,42.86) (A,44.16)  three:  14.691207985296362
eventType:  D1  time:  42.86  N:  4 N1

eventType:  A  time:  91.69  N:  6 N1:  1 N2:  1  numArr:  159  TotalTimeS:  222.91  FEL:  (E,100.00) (D2,92.32) (D1,91.77) (A,91.97)  three:  35.090224317652414
eventType:  D1  time:  91.77  N:  5 N1:  1 N2:  1  numArr:  159  TotalTimeS:  223.38  FEL:  (E,100.00) (D2,92.32) (A,91.97) (D1,92.06)  three:  35.16870237165881
eventType:  A  time:  91.97  N:  6 N1:  1 N2:  1  numArr:  160  TotalTimeS:  224.37  FEL:  (E,100.00) (D2,92.32) (D1,92.06) (A,92.79)  three:  35.36568915113084
eventType:  D1  time:  92.06  N:  5 N1:  1 N2:  1  numArr:  160  TotalTimeS:  224.92  FEL:  (E,100.00) (D2,92.32) (A,92.79) (D1,92.86)  three:  35.45758966203232
eventType:  D2  time:  92.32  N:  4 N1:  1 N2:  1  numArr:  160  TotalTimeS:  226.21  FEL:  (E,100.00) (A,92.79) (D1,92.86) (D2,93.74)  three:  35.715981102784625
eventType:  A  time:  92.79  N:  5 N1:  1 N2:  1  numArr:  161  TotalTimeS:  228.11  FEL:  (E,100.00) (D1,92.86) (D2,93.74) (A,93.46)  three:  36.19035289880823
eventType:  D1  time:  92.86 

eventType:  D2  time:  3.68  N:  2 N1:  1 N2:  1  numArr:  10  TotalTimeS:  10.34  FEL:  (E,100.00) (D1,5.09) (A,4.45) (D2,5.07)  three:  2.2757217778624543
eventType:  A  time:  4.45  N:  3 N1:  1 N2:  1  numArr:  11  TotalTimeS:  11.88  FEL:  (E,100.00) (D1,5.09) (D2,5.07) (A,5.49)  three:  2.2757217778624543
eventType:  D2  time:  5.07  N:  2 N1:  1 N2:  1  numArr:  11  TotalTimeS:  13.74  FEL:  (E,100.00) (D1,5.09) (A,5.49) (D2,8.67)  three:  2.8983978312809526
eventType:  D1  time:  5.09  N:  1 N1:  0 N2:  1  numArr:  11  TotalTimeS:  13.77  FEL:  (E,100.00) (A,5.49) (D2,8.67)  three:  2.8983978312809526
eventType:  A  time:  5.49  N:  2 N1:  1 N2:  1  numArr:  12  TotalTimeS:  14.17  FEL:  (E,100.00) (D2,8.67) (A,5.90) (D1,5.63)  three:  2.8983978312809526
eventType:  D1  time:  5.63  N:  1 N1:  0 N2:  1  numArr:  12  TotalTimeS:  14.45  FEL:  (E,100.00) (D2,8.67) (A,5.90)  three:  2.8983978312809526
eventType:  A  time:  5.90  N:  2 N1:  1 N2:  1  numArr:  13  TotalTimeS:  14.72

eventType:  A  time:  28.18  N:  5 N1:  1 N2:  1  numArr:  69  TotalTimeS:  102.04  FEL:  (E,100.00) (D1,28.57) (D2,28.27) (A,28.22)  three:  17.968620591927287
eventType:  A  time:  28.22  N:  6 N1:  1 N2:  1  numArr:  70  TotalTimeS:  102.21  FEL:  (E,100.00) (D1,28.57) (D2,28.27) (A,28.94)  three:  18.00274405428254
eventType:  D2  time:  28.27  N:  5 N1:  1 N2:  1  numArr:  70  TotalTimeS:  102.56  FEL:  (E,100.00) (D1,28.57) (A,28.94) (D2,29.28)  three:  18.06043560665479
eventType:  D1  time:  28.57  N:  4 N1:  1 N2:  1  numArr:  70  TotalTimeS:  104.02  FEL:  (E,100.00) (A,28.94) (D2,29.28) (D1,29.51)  three:  18.353237783810293
eventType:  A  time:  28.94  N:  5 N1:  1 N2:  1  numArr:  71  TotalTimeS:  105.53  FEL:  (E,100.00) (D2,29.28) (D1,29.51) (A,29.37)  three:  18.73016302853969
eventType:  D2  time:  29.28  N:  4 N1:  1 N2:  1  numArr:  71  TotalTimeS:  107.22  FEL:  (E,100.00) (D1,29.51) (A,29.37) (D2,29.54)  three:  19.069658873081575
eventType:  A  time:  29.37  N:  5

...

eventType:  D1  time:  7.16  N:  4 N1:  1 N2:  1  numArr:  21  TotalTimeS:  19.63  FEL:  (E,100.00) (D2,7.49) (A,7.61) (D1,7.35)  three:  3.549236193258235
eventType:  D1  time:  7.35  N:  3 N1:  1 N2:  1  numArr:  21  TotalTimeS:  20.39  FEL:  (E,100.00) (D2,7.49) (A,7.61) (D1,7.68)  three:  3.7398115847110702
eventType:  D2  time:  7.49  N:  2 N1:  1 N2:  1  numArr:  21  TotalTimeS:  20.82  FEL:  (E,100.00) (A,7.61) (D1,7.68) (D2,7.58)  three:  3.8845025671185924
eventType:  D2  time:  7.58  N:  1 N1:  1 N2:  0  numArr:  21  TotalTimeS:  21.00  FEL:  (E,100.00) (A,7.61) (D1,7.68)  three:  3.8845025671185924
eventType:  A  time:  7.61  N:  2 N1:  1 N2:  1  numArr:  22  TotalTimeS:  21.03  FEL:  (E,100.00) (D1,7.68) (A,7.70) (D2,8.39)  three:  3.8845025671185924
eventType:  D1  time:  7.68  N:  1 N1:  0 N2:  1  numArr:  22  TotalTimeS:  21.17  FEL:  (E,100.00) (A,7.70) (D2,8.39)  three:  3.8845025671185924
eventType:  A  time:  7.70  N:  2 N1:  1 N2:  1  numArr:  23  TotalTimeS:  21.19

eventType:  A  time:  24.38  N:  1 N1:  1 N2:  0  numArr:  62  TotalTimeS:  78.69  FEL:  (E,100.00) (A,26.55) (D1,24.65)  three:  15.452704171411288
eventType:  D1  time:  24.65  N:  0 N1:  0 N2:  0  numArr:  62  TotalTimeS:  78.96  FEL:  (E,100.00) (A,26.55)  three:  15.452704171411288
eventType:  A  time:  26.55  N:  1 N1:  1 N2:  0  numArr:  63  TotalTimeS:  78.96  FEL:  (E,100.00) (A,26.57) (D1,26.88)  three:  15.452704171411288
eventType:  A  time:  26.57  N:  2 N1:  1 N2:  1  numArr:  64  TotalTimeS:  78.97  FEL:  (E,100.00) (D1,26.88) (A,27.06) (D2,27.14)  three:  15.452704171411288
eventType:  D1  time:  26.88  N:  1 N1:  0 N2:  1  numArr:  64  TotalTimeS:  79.61  FEL:  (E,100.00) (A,27.06) (D2,27.14)  three:  15.452704171411288
eventType:  A  time:  27.06  N:  2 N1:  1 N2:  1  numArr:  65  TotalTimeS:  79.79  FEL:  (E,100.00) (D2,27.14) (A,27.10) (D1,27.36)  three:  15.452704171411288
eventType:  A  time:  27.10  N:  3 N1:  1 N2:  1  numArr:  66  TotalTimeS:  79.86  FEL:  (E,1

eventType:  D2  time:  51.07  N:  5 N1:  1 N2:  1  numArr:  110  TotalTimeS:  141.91  FEL:  (E,100.00) (D1,52.44) (A,51.45) (D2,52.27)  three:  26.373765829388535
eventType:  A  time:  51.45  N:  6 N1:  1 N2:  1  numArr:  111  TotalTimeS:  143.84  FEL:  (E,100.00) (D1,52.44) (D2,52.27) (A,51.88)  three:  26.759637905134067
eventType:  A  time:  51.88  N:  7 N1:  1 N2:  1  numArr:  112  TotalTimeS:  146.38  FEL:  (E,100.00) (D1,52.44) (D2,52.27) (A,51.95)  three:  27.18267982952482
eventType:  A  time:  51.95  N:  8 N1:  1 N2:  1  numArr:  113  TotalTimeS:  146.89  FEL:  (E,100.00) (D1,52.44) (D2,52.27) (A,52.67)  three:  27.255652883799154
eventType:  D2  time:  52.27  N:  7 N1:  1 N2:  1  numArr:  113  TotalTimeS:  149.43  FEL:  (E,100.00) (D1,52.44) (A,52.67) (D2,53.89)  three:  27.572743094017394
eventType:  D1  time:  52.44  N:  6 N1:  1 N2:  1  numArr:  113  TotalTimeS:  150.63  FEL:  (E,100.00) (A,52.67) (D2,53.89) (D1,53.03)  three:  27.744478726314423
eventType:  A  time:  52.6

eventType:  A  time:  68.11  N:  8 N1:  1 N2:  1  numArr:  152  TotalTimeS:  250.65  FEL:  (E,100.00) (D1,68.33) (D2,69.04) (A,68.74)  three:  43.41476062161769
eventType:  D1  time:  68.33  N:  7 N1:  1 N2:  1  numArr:  152  TotalTimeS:  252.45  FEL:  (E,100.00) (D2,69.04) (A,68.74) (D1,68.73)  three:  43.64000385381202
eventType:  D1  time:  68.73  N:  6 N1:  1 N2:  1  numArr:  152  TotalTimeS:  255.24  FEL:  (E,100.00) (D2,69.04) (A,68.74) (D1,69.14)  three:  44.037947811134984
eventType:  A  time:  68.74  N:  7 N1:  1 N2:  1  numArr:  153  TotalTimeS:  255.30  FEL:  (E,100.00) (D2,69.04) (D1,69.14) (A,70.44)  three:  44.04789362180611
eventType:  D2  time:  69.04  N:  6 N1:  1 N2:  1  numArr:  153  TotalTimeS:  257.42  FEL:  (E,100.00) (D1,69.14) (A,70.44) (D2,69.64)  three:  44.35138142260389
eventType:  D1  time:  69.14  N:  5 N1:  1 N2:  1  numArr:  153  TotalTimeS:  257.96  FEL:  (E,100.00) (A,70.44) (D2,69.64) (D1,69.29)  three:  44.44206314645457
eventType:  D1  time:  69.29 

eventType:  A  time:  88.01  N:  4 N1:  1 N2:  1  numArr:  197  TotalTimeS:  321.77  FEL:  (E,100.00) (D2,88.15) (D1,88.16) (A,89.49)  three:  57.65781279627049
eventType:  D2  time:  88.15  N:  3 N1:  1 N2:  1  numArr:  197  TotalTimeS:  322.32  FEL:  (E,100.00) (D1,88.16) (A,89.49) (D2,88.50)  three:  57.79480946156272
eventType:  D1  time:  88.16  N:  2 N1:  1 N2:  1  numArr:  197  TotalTimeS:  322.35  FEL:  (E,100.00) (A,89.49) (D2,88.50) (D1,88.79)  three:  57.805273919530876
eventType:  D2  time:  88.50  N:  1 N1:  1 N2:  0  numArr:  197  TotalTimeS:  323.03  FEL:  (E,100.00) (A,89.49) (D1,88.79)  three:  57.805273919530876
eventType:  D1  time:  88.79  N:  0 N1:  0 N2:  0  numArr:  197  TotalTimeS:  323.32  FEL:  (E,100.00) (A,89.49)  three:  57.805273919530876
eventType:  A  time:  89.49  N:  1 N1:  1 N2:  0  numArr:  198  TotalTimeS:  323.32  FEL:  (E,100.00) (A,90.39) (D1,89.86)  three:  57.805273919530876
eventType:  D1  time:  89.86  N:  0 N1:  0 N2:  0  numArr:  198  Total

In [276]:
import math
print("The confidence interval for threeOver is {0}".format(-np.std(th)*1.960/math.sqrt(20)+sum(th)/20)
      +" and {0}" .format(np.std(th)*1.960/math.sqrt(20)+ sum(th)/20))
print("The confidence interval for 1st server busy is {0}".format(-np.std(on)*1.960/math.sqrt(20)+sum(on)/20)
      +" and {0}" .format(np.std(on)*1.960/math.sqrt(20)+ sum(on)/20))
print("The confidence interval for both server busy is {0}".format(-np.std(tw)*1.960/math.sqrt(20)+sum(tw)/20)
      +" and {0}" .format(np.std(tw)*1.960/math.sqrt(20)+ sum(tw)/20))

The confidence interval for threeOver is 55.90034977032776 and 69.4730652239579
The confidence interval for 1st server busy is 82.23305667778675 and 88.18032672573423
The confidence interval for both server busy is 69.98218050383495 and 79.95954081880357
